In [80]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from numba import jit
from sklearn.model_selection import KFold
import multiprocessing as mp

In [60]:
data = np.loadtxt(('zipcombo.dat'))
num_labels = len(set(data[:,0]))

In [61]:

def poly_kernel(data, vec, d):
    '''
    INPUTS:
    data:       (#points,#pixels) matrix of data points (labels taken off)
    vec:        (#pixels,) vector to calculate kernel of
    d:          polynomial exponent
    OUTPUT:
    val:        (#points,) vector of kernel values with each training point
    '''

    val = (data @ vec.reshape(-1,1))**d
    return val.ravel()

def class_pred(data, vec, alphas, d):
    '''
    INPUTS:
    data:       (#points,#pixels) matrix of data points (labels taken off)
    vec:        (#pixels,) vector to calculate kernel of
    d:          polynomial exponent
    alphas:     (#labels,#points) matrix of values of alpha in the dual problem
    OUTPUT:
    preds:      (#labels,) vector of predictions for each label 
    '''
    kervals = poly_kernel(data,vec,d)
    preds = alphas @ kervals.reshape(-1,1)
    return preds.ravel()

def init_alphas(data):
    alphas = np.zeros((10,int(len(data))))
    return alphas

In [62]:
#training algorithm:
#have num_label individual classifiers which each classify whether it is or isnt that label
#on each point, we see how the classifier predicts, and update each classifier's coefficients which made a mistake
#the overall classifier's prediction is the classifier with the largest prediction 

def train_perceptron(data, alphas, d):
    '''
    INPUTS:
    data:        (#points,#pixels+1) matrix of data points
    alphas:      (#labels,#points) matrix of values of alpha in the dual problem
    d:           polynomial exponent
    OUTPUT:
    error_rate, alphas
    '''
    mistakes = 0
    #for each training point
    for i in range(len(data)):
        label = data[i,0]

        #obtain prediction made by each classifier
        preds = class_pred(data[:,1:],data[i,1:],alphas,d)
        preds_binary = np.where(preds <= 0, -1, 1)

        #check which classifier made a mistake
        truth_col = -np.ones(num_labels)
        truth_col[int(label)] = 1
        is_pred_wrong = (preds_binary != truth_col).astype(np.int32)          #a vector which has a 1 if the kth classifier was wrong

        #update alpha
        alphas[:,i] -= is_pred_wrong*preds_binary

        #add mistake
        if np.argmax(preds) != label:
            mistakes += 1
    
    error_rate = mistakes/len(data)
    return error_rate, alphas


#testing algorithm
def test_perceptron(train, test, alphas, d, calc_conf=False):
    '''

    '''
    mistakes = 0
    conf_mat = np.zeros((10,10))
    
    for i in range(len(test)):
        label = test[i,0]
        preds = class_pred(train[:,1:],test[i,1:],alphas,d)
        if int(np.argmax(preds)) != int(label):
            mistakes += 1
            if calc_conf:
                conf_mat[int(label),int(np.argmax(preds))] += 1

    #normalise conf_mat
    if calc_conf:
        label_counts = np.bincount(test[:,0].astype(int),minlength=10)
        label_counts[label_counts==0] = 1
        label_counts = label_counts.reshape(-1,1)

        #turn it into a rate
        conf_mat = conf_mat / label_counts

    error_rate = mistakes/len(test)

    if calc_conf:
        return error_rate, conf_mat  
    else:
        return error_rate


In [63]:
# def demo(train, test, d):
#     alpha_list = init_alphas(train)
#     for i in range(3):  # 3 iterations chosen arbitrarily
#         mistakes, alpha_list = train_perceptron(train, alpha_list, d)
#         print(f"Training - epoch {i+1} with {mistakes} mistakes out of {len(train)} items.")
#         test_error, conf_mat = test_perceptron(train, test, alpha_list, d, calc_conf=True)
#         print(f"Testing - epoch {i+1} with a test error of {test_error*100:.3f}%.")

# train_indices = np.arange(8000)
# train_data = data[train_indices,:]
# test_data = data[-train_indices,:]

# demo(train_data,test_data,d=3)        

1. Testing values of d

In [70]:
random.seed(1)

def train_test(data, d):
    shuffled_data = np.random.permutation(data)

    split_idx = int(len(data) * 0.8)

    train = shuffled_data[:split_idx, :]
    test = shuffled_data[split_idx:, :]

    alpha_list = init_alphas(train)

    train_error, alpha_list = train_perceptron(train, alpha_list, d)
    test_error = test_perceptron(train, test, alpha_list, d)

    return train_error, test_error

#results stored: dim1=d, dim2=run, dim3=train/test
results = np.zeros((7,20,2))

for d in range(7):
    for i in range(20):
        print(f'Calculating run {i+1} for power {d+1}.')
        results[d,i,0], results[d,i,1] = train_test(data,d+1) 

np.save('results1.npy', results)

Calculating run 1 for power 1.
Calculating run 2 for power 1.
Calculating run 3 for power 1.
Calculating run 4 for power 1.
Calculating run 5 for power 1.
Calculating run 6 for power 1.
Calculating run 7 for power 1.
Calculating run 8 for power 1.
Calculating run 9 for power 1.
Calculating run 10 for power 1.
Calculating run 11 for power 1.
Calculating run 12 for power 1.
Calculating run 13 for power 1.
Calculating run 14 for power 1.
Calculating run 15 for power 1.
Calculating run 16 for power 1.
Calculating run 17 for power 1.
Calculating run 18 for power 1.
Calculating run 19 for power 1.
Calculating run 20 for power 1.
Calculating run 1 for power 2.
Calculating run 2 for power 2.
Calculating run 3 for power 2.
Calculating run 4 for power 2.
Calculating run 5 for power 2.
Calculating run 6 for power 2.
Calculating run 7 for power 2.
Calculating run 8 for power 2.
Calculating run 9 for power 2.
Calculating run 10 for power 2.
Calculating run 11 for power 2.
Calculating run 12 for pow

In [71]:
results_loaded = np.load('results1.npy')

In [72]:
results_loaded

array([[[0.1469481 , 0.12473118],
        [0.14869589, 0.10752688],
        [0.1445281 , 0.15591398],
        [0.14708255, 0.09623656],
        [0.14708255, 0.09032258],
        [0.14748588, 0.12043011],
        [0.14775477, 0.09516129],
        [0.14815811, 0.1016129 ],
        [0.14923366, 0.10806452],
        [0.14923366, 0.11182796],
        [0.14909922, 0.11989247],
        [0.14869589, 0.11236559],
        [0.14493143, 0.09139785],
        [0.14479699, 0.12580645],
        [0.14802366, 0.11612903],
        [0.14278032, 0.15215054],
        [0.1418392 , 0.10967742],
        [0.14049476, 0.1344086 ],
        [0.14210809, 0.13817204],
        [0.14197365, 0.18978495]],

       [[0.09290132, 0.06612903],
        [0.09209465, 0.05913978],
        [0.09142243, 0.05752688],
        [0.08927131, 0.06290323],
        [0.0946491 , 0.06129032],
        [0.0946491 , 0.05053763],
        [0.09128798, 0.06129032],
        [0.09680022, 0.06290323],
        [0.09438021, 0.06021505],
        [0.0

2. Cross-validation and 3. Confusion matrix

In [82]:
%timeit

def cross_val(data, d, k=5):
    kf = KFold(n_splits=k)
    score = np.zeros(k)
    
    for i, (train_index, test_index) in enumerate(kf.split(data)):
        train_cv = data[train_index,:]
        test_cv = data[test_index,:]

        alpha_cv = init_alphas(train_cv)
            
        _, alpha_cv = train_perceptron(train_cv, alpha_cv, d+1)
        test_error = test_perceptron(train_cv, test_cv, alpha_cv, d+1)

        score[i] = test_error
    
    return score.mean()
     

def train_with_d_star(data):
    #split training and test
    shuffled_data = np.random.permutation(data)

    split_idx = int(len(data) * 0.8)

    train = shuffled_data[:split_idx, :]
    test = shuffled_data[split_idx:, :]



    params = [(train, d) for d in range(1, 8)]

    # Parallelize the cross-validation for different d values
    with mp.Pool() as pool:
        test_score = np.array(pool.starmap(cross_val, params))




    # #cross validate to find best d
    # test_score = np.array([cross_val(train, d) for d in range(1,8)])
    
    d_star = np.argmin(test_score) + 1

    #train perceptron with d_star
    alpha_list = init_alphas(train)

    _, alpha_list = train_perceptron(train, alpha_list, d_star)
    test_error, conf_mat = test_perceptron(train, test, alpha_list, d_star, calc_conf=True)

    return d_star, test_error, conf_mat


with mp.Pool() as pool:
    results = pool.map(train_with_d_star, [data]*20)

    # Initialize arrays to store the results
d_star_list = np.zeros(20)
test_error_list = np.zeros(20)
conf_mat_list = np.zeros((20,10,10))  # Adjust dimensions as necessary

    # Unpack and store the results in the arrays
for i, (d_star, test_error, conf_mat) in enumerate(results):
    d_star_list[i] = d_star
    test_error_list[i] = test_error
    conf_mat_list[i] = conf_mat

    # Save the results
np.save('d_star_list2.npy', d_star_list)
np.save('test_error_list2.npy', test_error_list)
np.save('conf_mat_list2.npy', conf_mat_list)






# d_star_list = np.zeros(20)
# test_error_list = np.zeros(20)
# conf_mat_list = np.zeros((20,10,10))
        
# for i in range(20):
#     d_star, test_error, conf_mat = train_with_d_star(data)
#     d_star_list[i] = d_star
#     test_error_list[i] = test_error
#     conf_mat_list[i,:,:] = conf_mat


# np.save('d_star_list2.npy', d_star_list)
# np.save('test_error_list2.npy', test_error_list)
# np.save('conf_mat_list2.npy', conf_mat_list)

    